## Day 82 Lecture 1 Assignment

In this assignment, we will learn about activation functions. We will create a neural network and measure the model's performance using different activations.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


We will import the famous titanic dataset below and produce a neural network that will predict the chance of survival for a passenger.

In [2]:
titanic = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/titanic.csv')

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We'll perform some feature engineering

Let's start by keeping only the columns we'd like to use for our analysis. Keep only the columns: Survived, Pclass, Sex, SibSp, Parch, and Embarked

In [4]:
# Answer below
an_cols = ['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
model_df = titanic[an_cols]
model_df

,Survived,Pclass,Sex,SibSp,Parch,Embarked
0,0,3,male,1,0,S
1,1,1,female,1,0,C
2,1,3,female,0,0,S
3,1,1,female,1,0,S
4,0,3,male,0,0,S
...,...,...,...,...,...,...
886,0,2,male,0,0,S
887,1,1,female,0,0,S
888,0,3,female,1,2,S
889,1,1,male,0,0,C


Now examine how many rows contain missing data. Given how much missing data we have, should we remove the column with the most missing data, or remove all rows containing missing data? Do what you think is best.

In [5]:
# Answer below: 
model_df.isna().mean()

Survived    0.000000
Pclass      0.000000
Sex         0.000000
SibSp       0.000000
Parch       0.000000
Embarked    0.002245
dtype: float64

In [6]:
model_df = model_df.dropna()
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  889 non-null    int64 
 1   Pclass    889 non-null    int64 
 2   Sex       889 non-null    object
 3   SibSp     889 non-null    int64 
 4   Parch     889 non-null    int64 
 5   Embarked  889 non-null    object
dtypes: int64(4), object(2)
memory usage: 48.6+ KB


Now we'll create a one hot encoding of the variables Pclass, sex, and Embarked

In [7]:
# Answer below:
dummy_df = pd.get_dummies(model_df, columns=['Pclass', 'Sex', 'Embarked'], drop_first=True)
dummy_df

,Survived,SibSp,Parch,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0,1,0,0,1,1,0,1
1,1,1,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1
3,1,1,0,0,0,0,0,1
4,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...
886,0,0,0,1,0,1,0,1
887,1,0,0,0,0,0,0,1
888,0,1,2,0,1,0,0,1
889,1,0,0,0,0,1,0,0


Split the data into train and test. 20% of the data should be set aside for testing. Use Survived as your target variable.

In [8]:
# Answer below
X = dummy_df.drop(columns=['Survived'])
y = dummy_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=66)

At this point, we are ready to create a model. Import `Sequential` and `Dense` from Keras

In [9]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Create a model with 5 layers. The first layer should be a dense layer that receives the input, the last layer should be of size 1. You determine the remaining layer sizes.

Use a tanh activation for the output layer.

In [10]:
# Answer below
model = Sequential()

# first dense layer
model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))

# hidden layers
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))

# output dense layer
model.add(Dense(1, activation='tanh'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 64        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 7         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


Compile the model using the adam optimizer, binary crossentropy loss, and the accuracy metric.

Fit the model using a batch size of 80 over 200 epochs.

In [12]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, epochs=500, batch_size=80, validation_data=(X_test, y_test))

Epoch 1/500
9/9 [==============================] - 0s 26ms/step - loss: 3.9875 - accuracy: 0.6259 - val_loss: 4.1910 - val_accuracy: 0.5843
Epoch 2/500
9/9 [==============================] - 0s 3ms/step - loss: 3.3001 - accuracy: 0.6259 - val_loss: 2.4741 - val_accuracy: 0.5843
Epoch 3/500
9/9 [==============================] - 0s 4ms/step - loss: 2.0609 - accuracy: 0.6259 - val_loss: 1.6913 - val_accuracy: 0.5843
Epoch 4/500
9/9 [==============================] - 0s 4ms/step - loss: 1.5542 - accuracy: 0.6259 - val_loss: 1.2684 - val_accuracy: 0.5843
Epoch 5/500
9/9 [==============================] - 0s 3ms/step - loss: 1.2099 - accuracy: 0.6259 - val_loss: 1.1241 - val_accuracy: 0.5843
Epoch 6/500
9/9 [==============================] - 0s 4ms/step - loss: 1.0776 - accuracy: 0.6259 - val_loss: 1.0827 - val_accuracy: 0.5843
Epoch 7/500
9/9 [==============================] - 0s 4ms/step - loss: 0.9991 - accuracy: 0.6259 - val_loss: 1.0553 - val_accuracy: 0.5843
Epoch 8/500
9/9 [=========

9/9 [==============================] - 0s 3ms/step - loss: 0.6213 - accuracy: 0.6231 - val_loss: 0.6612 - val_accuracy: 0.5787
Epoch 60/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6182 - accuracy: 0.6231 - val_loss: 0.6570 - val_accuracy: 0.5787
Epoch 61/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6149 - accuracy: 0.6231 - val_loss: 0.6531 - val_accuracy: 0.5843
Epoch 62/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6114 - accuracy: 0.6231 - val_loss: 0.6489 - val_accuracy: 0.5787
Epoch 63/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6080 - accuracy: 0.6245 - val_loss: 0.6450 - val_accuracy: 0.5843
Epoch 64/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6048 - accuracy: 0.6259 - val_loss: 0.6411 - val_accuracy: 0.5843
Epoch 65/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6016 - accuracy: 0.6259 - val_loss: 0.6370 - val_accuracy: 0.5843
Epoch 66/500
9/9 [===============

9/9 [==============================] - 0s 3ms/step - loss: 0.4925 - accuracy: 0.7848 - val_loss: 0.4890 - val_accuracy: 0.8034
Epoch 118/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4914 - accuracy: 0.7862 - val_loss: 0.4866 - val_accuracy: 0.8034
Epoch 119/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4902 - accuracy: 0.7876 - val_loss: 0.4847 - val_accuracy: 0.8034
Epoch 120/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4891 - accuracy: 0.7876 - val_loss: 0.4831 - val_accuracy: 0.8034
Epoch 121/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4878 - accuracy: 0.7876 - val_loss: 0.4810 - val_accuracy: 0.8034
Epoch 122/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4865 - accuracy: 0.7890 - val_loss: 0.4783 - val_accuracy: 0.8034
Epoch 123/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4853 - accuracy: 0.7890 - val_loss: 0.4763 - val_accuracy: 0.8034
Epoch 124/500
9/9 [========

9/9 [==============================] - 0s 4ms/step - loss: 0.4616 - accuracy: 0.7947 - val_loss: 0.4339 - val_accuracy: 0.8146
Epoch 176/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4611 - accuracy: 0.7947 - val_loss: 0.4332 - val_accuracy: 0.8146
Epoch 177/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4612 - accuracy: 0.7947 - val_loss: 0.4330 - val_accuracy: 0.8146
Epoch 178/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4598 - accuracy: 0.7947 - val_loss: 0.4322 - val_accuracy: 0.8146
Epoch 179/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4595 - accuracy: 0.7932 - val_loss: 0.4317 - val_accuracy: 0.8146
Epoch 180/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4590 - accuracy: 0.7947 - val_loss: 0.4317 - val_accuracy: 0.8146
Epoch 181/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4592 - accuracy: 0.7947 - val_loss: 0.4309 - val_accuracy: 0.8146
Epoch 182/500
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4419 - accuracy: 0.8143 - val_loss: 0.4203 - val_accuracy: 0.8090
Epoch 234/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4418 - accuracy: 0.8143 - val_loss: 0.4188 - val_accuracy: 0.8090
Epoch 235/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4417 - accuracy: 0.8143 - val_loss: 0.4170 - val_accuracy: 0.8090
Epoch 236/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4417 - accuracy: 0.8115 - val_loss: 0.4169 - val_accuracy: 0.7978
Epoch 237/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4421 - accuracy: 0.8143 - val_loss: 0.4193 - val_accuracy: 0.8034
Epoch 238/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4416 - accuracy: 0.8115 - val_loss: 0.4188 - val_accuracy: 0.8034
Epoch 239/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4409 - accuracy: 0.8115 - val_loss: 0.4168 - val_accuracy: 0.7921
Epoch 240/500
9/9 [========

9/9 [==============================] - 0s 4ms/step - loss: 0.4349 - accuracy: 0.8073 - val_loss: 0.4187 - val_accuracy: 0.8146
Epoch 292/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4359 - accuracy: 0.8087 - val_loss: 0.4144 - val_accuracy: 0.8090
Epoch 293/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4347 - accuracy: 0.8087 - val_loss: 0.4133 - val_accuracy: 0.7978
Epoch 294/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4339 - accuracy: 0.8087 - val_loss: 0.4160 - val_accuracy: 0.8034
Epoch 295/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4340 - accuracy: 0.8101 - val_loss: 0.4151 - val_accuracy: 0.8090
Epoch 296/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4347 - accuracy: 0.8073 - val_loss: 0.4129 - val_accuracy: 0.7978
Epoch 297/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4339 - accuracy: 0.8087 - val_loss: 0.4153 - val_accuracy: 0.8090
Epoch 298/500
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4290 - accuracy: 0.8115 - val_loss: 0.4103 - val_accuracy: 0.7921
Epoch 350/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4293 - accuracy: 0.8073 - val_loss: 0.4109 - val_accuracy: 0.7921
Epoch 351/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4317 - accuracy: 0.8129 - val_loss: 0.4124 - val_accuracy: 0.8090
Epoch 352/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4295 - accuracy: 0.8073 - val_loss: 0.4107 - val_accuracy: 0.7921
Epoch 353/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4285 - accuracy: 0.8073 - val_loss: 0.4108 - val_accuracy: 0.8034
Epoch 354/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4287 - accuracy: 0.8087 - val_loss: 0.4113 - val_accuracy: 0.8090
Epoch 355/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4295 - accuracy: 0.8073 - val_loss: 0.4124 - val_accuracy: 0.8090
Epoch 356/500
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4251 - accuracy: 0.8101 - val_loss: 0.4071 - val_accuracy: 0.8034
Epoch 407/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4262 - accuracy: 0.8059 - val_loss: 0.4068 - val_accuracy: 0.8034
Epoch 408/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4248 - accuracy: 0.8129 - val_loss: 0.4071 - val_accuracy: 0.8034
Epoch 409/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4247 - accuracy: 0.8101 - val_loss: 0.4071 - val_accuracy: 0.8034
Epoch 410/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4253 - accuracy: 0.8129 - val_loss: 0.4073 - val_accuracy: 0.8034
Epoch 411/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4241 - accuracy: 0.8129 - val_loss: 0.4073 - val_accuracy: 0.8034
Epoch 412/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4252 - accuracy: 0.8129 - val_loss: 0.4072 - val_accuracy: 0.8034
Epoch 413/500
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4206 - accuracy: 0.8158 - val_loss: 0.4017 - val_accuracy: 0.8090
Epoch 465/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4205 - accuracy: 0.8129 - val_loss: 0.4012 - val_accuracy: 0.8090
Epoch 466/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4214 - accuracy: 0.8158 - val_loss: 0.4025 - val_accuracy: 0.8090
Epoch 467/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4202 - accuracy: 0.8158 - val_loss: 0.4022 - val_accuracy: 0.8090
Epoch 468/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4213 - accuracy: 0.8158 - val_loss: 0.4050 - val_accuracy: 0.8090
Epoch 469/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4222 - accuracy: 0.8172 - val_loss: 0.4022 - val_accuracy: 0.8090
Epoch 470/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4232 - accuracy: 0.8158 - val_loss: 0.4010 - val_accuracy: 0.8090
Epoch 471/500
9/9 [========

Redefine the model using a sigmoid activation for the last layer. What is the difference in accuracy.

In [14]:
# Answer below
model = Sequential()

# first dense layer
model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))

# hidden layers
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))

# output dense layer
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 8)                 64        
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


Compile the model using the adam optimizer, binary crossentropy loss, and the accuracy metric.

Fit the model using a batch size of 80 over 200 epochs.

In [16]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=500, batch_size=80, validation_data=(X_test, y_test))

Epoch 1/500
9/9 [==============================] - 0s 6ms/step - loss: 0.4262 - accuracy: 0.8059 - val_loss: 0.3832 - val_accuracy: 0.8146
Epoch 2/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4249 - accuracy: 0.8059 - val_loss: 0.3854 - val_accuracy: 0.8146
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4250 - accuracy: 0.8045 - val_loss: 0.3843 - val_accuracy: 0.8146
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4253 - accuracy: 0.8059 - val_loss: 0.3817 - val_accuracy: 0.8202
Epoch 5/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4249 - accuracy: 0.8059 - val_loss: 0.3837 - val_accuracy: 0.8146
Epoch 6/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4239 - accuracy: 0.8073 - val_loss: 0.3850 - val_accuracy: 0.8146
Epoch 7/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4244 - accuracy: 0.8059 - val_loss: 0.3841 - val_accuracy: 0.8146
Epoch 8/500
9/9 [==========

9/9 [==============================] - 0s 4ms/step - loss: 0.4217 - accuracy: 0.8087 - val_loss: 0.3829 - val_accuracy: 0.8202
Epoch 60/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4216 - accuracy: 0.8101 - val_loss: 0.3847 - val_accuracy: 0.8146
Epoch 61/500
9/9 [==============================] - 0s 5ms/step - loss: 0.4219 - accuracy: 0.8073 - val_loss: 0.3846 - val_accuracy: 0.8202
Epoch 62/500
9/9 [==============================] - 0s 5ms/step - loss: 0.4216 - accuracy: 0.8073 - val_loss: 0.3845 - val_accuracy: 0.8146
Epoch 63/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4214 - accuracy: 0.8087 - val_loss: 0.3836 - val_accuracy: 0.8202
Epoch 64/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4213 - accuracy: 0.8087 - val_loss: 0.3848 - val_accuracy: 0.8146
Epoch 65/500
9/9 [==============================] - 0s 4ms/step - loss: 0.4219 - accuracy: 0.8101 - val_loss: 0.3839 - val_accuracy: 0.8146
Epoch 66/500
9/9 [===============

9/9 [==============================] - 0s 3ms/step - loss: 0.4192 - accuracy: 0.8115 - val_loss: 0.3846 - val_accuracy: 0.8202
Epoch 118/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4191 - accuracy: 0.8143 - val_loss: 0.3838 - val_accuracy: 0.8202
Epoch 119/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4196 - accuracy: 0.8129 - val_loss: 0.3853 - val_accuracy: 0.8202
Epoch 120/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4189 - accuracy: 0.8129 - val_loss: 0.3835 - val_accuracy: 0.8202
Epoch 121/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4195 - accuracy: 0.8129 - val_loss: 0.3846 - val_accuracy: 0.8202
Epoch 122/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4190 - accuracy: 0.8115 - val_loss: 0.3839 - val_accuracy: 0.8202
Epoch 123/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4188 - accuracy: 0.8115 - val_loss: 0.3838 - val_accuracy: 0.8202
Epoch 124/500
9/9 [========

Epoch 175/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4160 - accuracy: 0.8158 - val_loss: 0.3850 - val_accuracy: 0.8202
Epoch 176/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4170 - accuracy: 0.8143 - val_loss: 0.3836 - val_accuracy: 0.8146
Epoch 177/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4157 - accuracy: 0.8172 - val_loss: 0.3838 - val_accuracy: 0.8146
Epoch 178/500
9/9 [==============================] - 0s 5ms/step - loss: 0.4161 - accuracy: 0.8158 - val_loss: 0.3837 - val_accuracy: 0.8202
Epoch 179/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4159 - accuracy: 0.8172 - val_loss: 0.3831 - val_accuracy: 0.8146
Epoch 180/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4158 - accuracy: 0.8172 - val_loss: 0.3831 - val_accuracy: 0.8146
Epoch 181/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4168 - accuracy: 0.8158 - val_loss: 0.3859 - val_accuracy: 0.8258
Epoch 182/500

9/9 [==============================] - 0s 3ms/step - loss: 0.4147 - accuracy: 0.8172 - val_loss: 0.3857 - val_accuracy: 0.8146
Epoch 234/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4136 - accuracy: 0.8172 - val_loss: 0.3833 - val_accuracy: 0.8146
Epoch 235/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4131 - accuracy: 0.8172 - val_loss: 0.3836 - val_accuracy: 0.8146
Epoch 236/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4147 - accuracy: 0.8172 - val_loss: 0.3822 - val_accuracy: 0.8146
Epoch 237/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4138 - accuracy: 0.8186 - val_loss: 0.3849 - val_accuracy: 0.8258
Epoch 238/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4133 - accuracy: 0.8172 - val_loss: 0.3835 - val_accuracy: 0.8146
Epoch 239/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4141 - accuracy: 0.8129 - val_loss: 0.3820 - val_accuracy: 0.8146
Epoch 240/500
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4105 - accuracy: 0.8186 - val_loss: 0.3817 - val_accuracy: 0.8258
Epoch 292/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4105 - accuracy: 0.8186 - val_loss: 0.3818 - val_accuracy: 0.8258
Epoch 293/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4101 - accuracy: 0.8186 - val_loss: 0.3805 - val_accuracy: 0.8258
Epoch 294/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4104 - accuracy: 0.8186 - val_loss: 0.3811 - val_accuracy: 0.8258
Epoch 295/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4109 - accuracy: 0.8186 - val_loss: 0.3810 - val_accuracy: 0.8258
Epoch 296/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4109 - accuracy: 0.8186 - val_loss: 0.3845 - val_accuracy: 0.8258
Epoch 297/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4102 - accuracy: 0.8186 - val_loss: 0.3799 - val_accuracy: 0.8258
Epoch 298/500
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4085 - accuracy: 0.8228 - val_loss: 0.3785 - val_accuracy: 0.8315
Epoch 350/500
9/9 [==============================] - 0s 2ms/step - loss: 0.4090 - accuracy: 0.8186 - val_loss: 0.3810 - val_accuracy: 0.8258
Epoch 351/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4089 - accuracy: 0.8214 - val_loss: 0.3793 - val_accuracy: 0.8315
Epoch 352/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4074 - accuracy: 0.8228 - val_loss: 0.3806 - val_accuracy: 0.8258
Epoch 353/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4069 - accuracy: 0.8228 - val_loss: 0.3788 - val_accuracy: 0.8315
Epoch 354/500
9/9 [==============================] - 0s 2ms/step - loss: 0.4074 - accuracy: 0.8228 - val_loss: 0.3791 - val_accuracy: 0.8315
Epoch 355/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4070 - accuracy: 0.8228 - val_loss: 0.3786 - val_accuracy: 0.8315
Epoch 356/500
9/9 [========

Epoch 407/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4047 - accuracy: 0.8228 - val_loss: 0.3784 - val_accuracy: 0.8258
Epoch 408/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4048 - accuracy: 0.8228 - val_loss: 0.3790 - val_accuracy: 0.8258
Epoch 409/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4053 - accuracy: 0.8143 - val_loss: 0.3802 - val_accuracy: 0.8315
Epoch 410/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4070 - accuracy: 0.8228 - val_loss: 0.3795 - val_accuracy: 0.8258
Epoch 411/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4049 - accuracy: 0.8228 - val_loss: 0.3804 - val_accuracy: 0.8315
Epoch 412/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4050 - accuracy: 0.8228 - val_loss: 0.3792 - val_accuracy: 0.8258
Epoch 413/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4050 - accuracy: 0.8228 - val_loss: 0.3792 - val_accuracy: 0.8315
Epoch 414/500

9/9 [==============================] - 0s 3ms/step - loss: 0.4013 - accuracy: 0.8200 - val_loss: 0.3781 - val_accuracy: 0.8258
Epoch 466/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4025 - accuracy: 0.8214 - val_loss: 0.3783 - val_accuracy: 0.8258
Epoch 467/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4013 - accuracy: 0.8214 - val_loss: 0.3784 - val_accuracy: 0.8258
Epoch 468/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4019 - accuracy: 0.8228 - val_loss: 0.3783 - val_accuracy: 0.8258
Epoch 469/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4012 - accuracy: 0.8242 - val_loss: 0.3785 - val_accuracy: 0.8258
Epoch 470/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4033 - accuracy: 0.8256 - val_loss: 0.3801 - val_accuracy: 0.8427
Epoch 471/500
9/9 [==============================] - 0s 3ms/step - loss: 0.4012 - accuracy: 0.8242 - val_loss: 0.3778 - val_accuracy: 0.8258
Epoch 472/500
9/9 [========